Problem 1: The Plant Growth Evaluator

Question Summary:
You need to check:

Average height > 50 cm

Difference between tallest and shortest < 20 cm

Then print:

Both true → "Plants are healthy and consistent!"

Only first true → "Plants are growing well but unevenly!"

Otherwise → "Plants need better growth conditions."

Example:
Input: [52, 48, 55, 51]
Output: Plants are healthy and consistent!


In [1]:
def evaluate_growth(heights):
    avg_height = sum(heights) / len(heights)
    height_diff = max(heights) - min(heights)
    
    if avg_height > 50 and height_diff < 20:
        print("Plants are healthy and consistent!")
    elif avg_height > 50:
        print("Plants are growing well but unevenly!")
    else:
        print("Plants need better growth conditions.")

# --- Input Option ---
# Uncomment below lines to take input manually
# heights = list(map(int, input("Enter heights of 4 plants (space-separated): ").split()))
# evaluate_growth(heights)

# --- Example Test Call ---
evaluate_growth([52, 48, 55, 51])


Plants are healthy and consistent!


Problem 02: The Robot Task Scheduler
🧾 Question Summary

A robot must complete 3 tasks: cleaning, sorting, and charging.
Each task takes a certain amount of time (in minutes), and the battery limit is 100 minutes.

The program should:

Take the times for all three tasks as input.

Calculate the total time.

If total ≤ 100 → print "All tasks completed successfully!"

If total > 100 → print:

The total time

Which tasks were completed before the battery ran out

Which tasks could not be completed (based on order)

How much extra time was needed

Example:
Input → cleaning = 40, sorting = 30, charging = 50
Output:
Total time = 120 minutes
Tasks completed: cleaning, sorting
Task not completed: charging
Extra time needed: 20 minutes



In [2]:
def robot_task_scheduler(cleaning, sorting, charging):
    """Problem 2: Robot Task Scheduler"""
    tasks = [("cleaning", cleaning), ("sorting", sorting), ("charging", charging)]
    total_time = cleaning + sorting + charging

    print(f"Total time = {total_time} minutes")
    BATTERY = 100

    if total_time <= BATTERY:
        print("All tasks completed successfully!")
        return

    # Determine completed and incomplete tasks
    completed = []
    not_completed = []
    used_time = 0

    for name, t in tasks:
        if used_time + t <= BATTERY:
            used_time += t
            completed.append(name)
        else:
            not_completed.append(name)
            # Add remaining tasks as not completed
            idx = [n for n, _ in tasks].index(name)
            for later, _ in tasks[idx+1:]:
                not_completed.append(later)
            break

    if completed:
        print("Tasks completed: " + ", ".join(completed))
    else:
        print("Tasks completed: none")

    if len(not_completed) == 1:
        print("Task not completed: " + not_completed[0])
    else:
        print("Tasks not completed: " + ", ".join(not_completed))

    extra_time = total_time - BATTERY
    print(f"Extra time needed: {extra_time} minutes")

# --- Input Option ---
# Uncomment to take input manually:
# cleaning = int(input("Enter cleaning time (in minutes): "))
# sorting = int(input("Enter sorting time (in minutes): "))
# charging = int(input("Enter charging time (in minutes): "))
# robot_task_scheduler(cleaning, sorting, charging)

# --- Example Test Call ---
robot_task_scheduler(40, 30, 50)


Total time = 120 minutes
Tasks completed: cleaning, sorting
Task not completed: charging
Extra time needed: 20 minutes


You are given a graph representing a map of Romania, where nodes are cities and edges are roads with travel distances (in km). You need to determine the optimal route from Arad to Bucharest.

Given:

Each road has a distance (edge cost).

Each city has a heuristic value (straight-line distance to Bucharest).

Tasks:

Implement A* or Greedy Best First Search algorithm.

Find and print:

The final path from Arad to Bucharest.

The total cost of the path.

In [3]:
from queue import PriorityQueue

# Romania map graph with step costs
romania_map = {
    'Arad': {'Zerind': 75, 'Timisoara': 118, 'Sibiu': 140},
    'Zerind': {'Arad': 75, 'Oradea': 71},
    'Oradea': {'Zerind': 71, 'Sibiu': 151},
    'Timisoara': {'Arad': 118, 'Lugoj': 111},
    'Lugoj': {'Timisoara': 111, 'Mehadia': 70},
    'Mehadia': {'Lugoj': 70, 'Dobreta': 75},
    'Dobreta': {'Mehadia': 75, 'Craiova': 120},
    'Craiova': {'Dobreta': 120, 'Rimnicu Vilcea': 146, 'Pitesti': 138},
    'Sibiu': {'Arad': 140, 'Oradea': 151, 'Fagaras': 99, 'Rimnicu Vilcea': 80},
    'Rimnicu Vilcea': {'Sibiu': 80, 'Pitesti': 97, 'Craiova': 146},
    'Fagaras': {'Sibiu': 99, 'Bucharest': 211},
    'Pitesti': {'Rimnicu Vilcea': 97, 'Craiova': 138, 'Bucharest': 101},
    'Bucharest': {'Fagaras': 211, 'Pitesti': 101, 'Giurgiu': 90, 'Urziceni': 85},
    'Giurgiu': {'Bucharest': 90},
    'Urziceni': {'Bucharest': 85, 'Vaslui': 142, 'Hirsova': 98},
    'Vaslui': {'Urziceni': 142, 'Iasi': 92},
    'Iasi': {'Vaslui': 92, 'Neamt': 87},
    'Neamt': {'Iasi': 87},
    'Hirsova': {'Urziceni': 98, 'Eforie': 86},
    'Eforie': {'Hirsova': 86}
}

# Heuristic values (straight-line distance to Bucharest)
heuristic = {
    'Arad': 366, 'Bucharest': 0, 'Craiova': 160, 'Dobreta': 242, 'Eforie': 161,
    'Fagaras': 178, 'Giurgiu': 77, 'Hirsova': 151, 'Iasi': 226, 'Lugoj': 244,
    'Mehadia': 241, 'Neamt': 234, 'Oradea': 380, 'Pitesti': 98, 'Rimnicu Vilcea': 193,
    'Sibiu': 253, 'Timisoara': 329, 'Urziceni': 80, 'Vaslui': 199, 'Zerind': 374
}

def a_star_search(start, goal):
    open_set = PriorityQueue()
    open_set.put((0, start))
    
    came_from = {}
    g_score = {city: float('inf') for city in romania_map}
    g_score[start] = 0

    f_score = {city: float('inf') for city in romania_map}
    f_score[start] = heuristic[start]

    while not open_set.empty():
        _, current = open_set.get()

        if current == goal:
            # Reconstruct path
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            return path, g_score[goal]

        for neighbor, distance in romania_map[current].items():
            tentative_g = g_score[current] + distance
            if tentative_g < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g
                f_score[neighbor] = tentative_g + heuristic[neighbor]
                open_set.put((f_score[neighbor], neighbor))

    return None, float('inf')

# Run A* from Arad to Bucharest
path, cost = a_star_search('Arad', 'Bucharest')

print("Optimal Path:", " → ".join(path))
print("Total Cost:", cost)


Optimal Path: Arad → Sibiu → Rimnicu Vilcea → Pitesti → Bucharest
Total Cost: 418
